### INST447 Project Part 2
#### Members: Alexander Chui and Matthew Chan

### What factors affect ratings for an anime?

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Importing the required libraries
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen 
import numpy as np
from requests import get

In [3]:
#libraries imported for handling rate limits and console formatting
from time import time, sleep
from random import randint
from warnings import warn
from IPython.core.display import clear_output

In [4]:
import regex as re

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis

import nltk; 

C:\Users\MaNinjaChan\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\MaNinjaChan\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:164: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar'

In [5]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [6]:
#Connecting to the URL on the web to access its content (Crunchyroll)
# html=urlopen('https://www.crunchyroll.com/fullmetal-alchemist-brotherhood/reviews/helpful/page1')
# html.status

In [7]:
#Connecting to the URL on the web to access its content (MAL)
html=urlopen('https://myanimelist.net/anime/31338/Hundred/reviews?p=1')
html.status

200

In [8]:
#Creating an instance of beautifulsoup object to begin web scraping
bs=BeautifulSoup(html.read(),'html.parser')

In [9]:
#Acessing the container objects with the anime info 
anime_review_containers = bs.find_all('div', {'class':{'borderDark'}})
print(len(anime_review_containers))

20


In [10]:
#Creating empty lists to store all the items 
review = []

In [11]:
# Preparing the monitoring of the loop
start_time = time()
request = 0

In [12]:
#Extracting each job name in loop
for page in range(1,3):
    # Make a get request
        #https://myanimelist.net/anime/genre/1/Action?page=1
        #https://myanimelist.net/anime/genre/1/Action?page=2
        response = get('https://myanimelist.net/anime/31338/Hundred/reviews?p='+ str(page))
        
#         # Pause the loop
#         sleep(randint(8,15))

#         # Monitor the requests
#         request += 1
#         elapsed_time = time() - start_time
#         print('Request:{}; Frequency: {} requests/s'.format(request, 
#                                                             request/elapsed_time))
#         clear_output(wait = True)

#         # Throw a warning for non-200 status codes
#         if response.status_code != 200:
#             warn('Request: {}; Status code: {}'.format(request, 
#                                                        response.status_code))

#         # Break the loop if the number of requests is greater than expected
#         if request > 100:
#             warn('Number of requests was greater than expected.')
#             break
        
        # Parse the content of the request with BeautifulSoup
        bs = BeautifulSoup(response.text, 'html.parser')
        
        #Acessing the container objects with the anime info 
        anime_review_containers = bs.find_all('div', {'class':{'borderDark'}})

        #Extracting all the elements 
        for container in anime_review_containers:
        # If the movie has Metascore, then extract:
        #  if container.find('div', {'class':{'scoremem'}})  is not None:
        
        # The name of anime
            try: 
                reviews = container.find('div', {'class':'spaceit textReadability word-break pt8 mt8'})
                review.append(reviews.text)
            except:
                review.append('Missing')

In [13]:
#Creating a pandas dataframe to store the objects extracted
anime_review = pd.DataFrame({'Review': review
})
print(anime_review.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  40 non-null     object
dtypes: object(1)
memory usage: 448.0+ bytes
None


In [14]:
# Display the top 5 rows of dataset
anime_review.head()

,Review
0,\n\n\n\nOverall\n5\n\n\nStory\n3\n\n\nAnimatio...
1,\n\n\n\nOverall\n2\n\n\nStory\n3\n\n\nAnimatio...
2,\n\n\n\nOverall\n8\n\n\nStory\n7\n\n\nAnimatio...
3,\n\n\n\nOverall\n7\n\n\nStory\n6\n\n\nAnimatio...
4,\n\n\n\nOverall\n6\n\n\nStory\n3\n\n\nAnimatio...


In [15]:
# Convert to list
data = anime_review['Review'].tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

print(data[0])

 Overall 5 Story 3 Animation 7 Sound 7 Character 5 Enjoyment 5 "They call it Hundred because you've seen this show hundreds of times already." I don't remember the name of the original user who said this but I do like that part in your review. If I had to say what Hundred's weakness is, its that the show lacks any uniqueness to help it stand out more from the rest of the magic school battle anime out there. It appears as if there is a really bad cycle in Japan right now where so many.. school/harem/sci-fi anime are taking over and in my opinion not too many of them are doing so well. I mean we've had asterisk wars, chivalry of a failed knight and many more similar every season and each title just leaves me with mixed feelings. Now the story is simple, straight-forward yet lacks uniqueness which I believe is something that is absolutely vital in every anime & manga. Hundred is literally copy & pasted meaning if you have watched at least 2 or three titles similar to this then you can alr

<>:5: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \s
<ipython-input-15-b412c0db5fd7>:5: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


In [16]:
# Clean and tokenize the reviews

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data_words = list(sent_to_words(data))

print(data_words[0])

['overall', 'story', 'animation', 'sound', 'character', 'enjoyment', 'they', 'call', 'it', 'hundred', 'because', 'you', 've', 'seen', 'this', 'show', 'hundreds', 'of', 'times', 'already', 'don', 'remember', 'the', 'name', 'of', 'the', 'original', 'user', 'who', 'said', 'this', 'but', 'do', 'like', 'that', 'part', 'in', 'your', 'review', 'if', 'had', 'to', 'say', 'what', 'hundred', 'weakness', 'is', 'its', 'that', 'the', 'show', 'lacks', 'any', 'uniqueness', 'to', 'help', 'it', 'stand', 'out', 'more', 'from', 'the', 'rest', 'of', 'the', 'magic', 'school', 'battle', 'anime', 'out', 'there', 'it', 'appears', 'as', 'if', 'there', 'is', 'really', 'bad', 'cycle', 'in', 'japan', 'right', 'now', 'where', 'so', 'many', 'school', 'harem', 'sci', 'fi', 'anime', 'are', 'taking', 'over', 'and', 'in', 'my', 'opinion', 'not', 'too', 'many', 'of', 'them', 'are', 'doing', 'so', 'well', 'mean', 'we', 've', 'had', 'asterisk', 'wars', 'chivalry', 'of', 'failed', 'knight', 'and', 'many', 'more', 'similar',

In [17]:
# Create bigrams, remove stop words and lemmatize the reviews

# Build the bigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

print(bigram_mod[data_words[4]])

['overall', 'story', 'animation_sound', 'character', 'enjoyment', 'story', 'there', 'not', 'much', 'of', 'story', 'it', 'basically', 'about', 'the', 'characters', 'protecting', 'the', 'ship', 'art', 'the', 'art', 'was', 'okay', 'but', 'couldn', 'help', 'but', 'notice', 'the', 'glaring', 'errors', 'in', 'the', 'animation', 'sometimes', 'it', 'was', 'stilted', 'and', 'lazy', 'for', 'example', 'there', 'are', 'times', 'when', 'someone', 'is', 'in', 'position', 'in', 'frame', 'then', 'they', 'cut', 'to', 'someone', 'right', 'next', 'to', 'them', 'and', 'then', 'they', 'cut', 'to', 'the', 'person', 'next', 'to', 'them', 'in', 'another', 'position', 'entirely', 'it', 'as', 'if', 'sometimes', 'they', 'didn', 'want', 'to', 'put', 'in', 'the', 'work', 'of', 'the', 'character', 'doing', 'fluid', 'animation', 'or', 'doing', 'something', 'in', 'one', 'frame', 'this', 'is', 'especially', 'noticeable', 'in', 'the', 'last', 'few', 'episodes', 'sound', 'the', 'music', 'was', 'good', 'especially', 'the

In [18]:
# Defining removing stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Defining making bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Defining lemmatization
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

['overall', 'story', 'character', 'enjoyment', 'call', 'see', 'show', 'hundred', 'time', 'already', 'remember', 'name', 'original', 'user', 'say', 'part', 'review', 'say', 'weakness', 'show', 'lack', 'uniqueness', 'help', 'stand', 'rest', 'magic', 'school', 'battle', 'anime', 'appear', 'really', 'bad', 'cycle', 'japan', 'right', 'many', 'school', 'harem', 'sci', 'fi', 'anime', 'take', 'opinion', 'many', 'well', 'mean', 'asterisk', 'war', 'chivalry', 'fail', 'knight', 'many', 'similar', 'season', 'title', 'leave', 'mixed', 'feeling', 'story', 'simple', 'straight', 'forward', 'yet', 'lack', 'uniqueness', 'believe', 'absolutely', 'vital', 'anime', 'manga', 'literally', 'copy', 'paste', 'meaning', 'watch', 'least', 'title', 'similar', 'already', 'predict', 'upcoming', 'event', 'relationship', 'character', 'addition', 'prediction', 'plot', 'event', 'go', 'take', 'place', 'certain', 'episode', 'sort', 'walk', 'park', 'really', 'try', 'criticize', 'jun', 'misaki', 'author', 'obviously', 'idea

In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 3), (8, 1), (9, 1), (10, 4), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 2), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 3), (24, 2), (25, 2), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 11), (34, 2), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 3), (42, 1), (43, 1), (44, 1), (45, 2), (46, 1), (47, 1), (48, 1), (49, 2), (50, 2), (51, 1), (52, 2), (53, 1), (54, 1), (55, 4), (56, 2), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2), (64, 1), (65, 3), (66, 2), (67, 1), (68, 1), (69, 1), (70, 4), (71, 1), (72, 1), (73, 6), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 3), (89, 1), (90, 1), (91, 3), (92, 1), (93, 4), (94, 1), (95, 1), (96, 1), (97, 2), (98, 1), (99, 1), (100, 7), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 2), (107, 1), (108, 1), (109, 2), (110, 2)

In [21]:
# Using LDA to develop a topic model with 3 models

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
# Print the KeywordS in the 3 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.025*"character" + 0.013*"good" + 0.012*"really" + 0.012*"feel" + 0.008*"anime" + 0.008*"show" + 0.008*"story" + 0.007*"episode" + 0.007*"series" + 0.007*"end"'), (1, '0.023*"anime" + 0.019*"watch" + 0.013*"harem" + 0.010*"character" + 0.010*"story" + 0.010*"enjoy" + 0.009*"overall" + 0.009*"mc" + 0.008*"kun" + 0.007*"plot"'), (2, '0.013*"character" + 0.013*"girl" + 0.011*"get" + 0.010*"hayato" + 0.010*"reason" + 0.009*"show" + 0.009*"anime" + 0.008*"guy" + 0.008*"story" + 0.008*"first"'), (3, '0.024*"show" + 0.021*"character" + 0.020*"anime" + 0.011*"story" + 0.010*"main" + 0.009*"well" + 0.008*"harem" + 0.008*"get" + 0.008*"fanservice" + 0.006*"school"'), (4, '0.029*"character" + 0.022*"show" + 0.016*"really" + 0.009*"get" + 0.009*"main" + 0.009*"actually" + 0.008*"even" + 0.007*"good" + 0.007*"time" + 0.006*"plot"'), (5, '0.024*"character" + 0.019*"anime" + 0.009*"main" + 0.008*"get" + 0.008*"time" + 0.006*"good" + 0.006*"story" + 0.006*"girl" + 0.006*"hayato" + 0.006*"watch"

In [22]:
# Generate a visualization of the topic models

import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.038773  0.010076       1        1  46.988901
4     -0.091124  0.040362       2        1  14.940746
3     -0.056979 -0.080059       3        1  10.655105
2      0.020842  0.118242       4        1   8.445044
5      0.039506  0.006906       5        1   7.227844
0     -0.042896 -0.038047       6        1   7.062390
1      0.083982 -0.033424       7        1   4.146114
6      0.085442 -0.024056       8        1   0.533856, topic_info=          Term        Freq       Total Category  logprob  loglift
10       anime  199.000000  199.000000  Default  30.0000  30.0000
33   character  276.000000  276.000000  Default  29.0000  29.0000
246      watch   79.000000   79.000000  Default  28.0000  28.0000
211       show  197.000000  197.000000  Default  27.0000  27.0000
225      story  144.000000  144.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
441      thing    0.329544   39.734766   Topic8  -5.3456   0.4405
225      story    0.457010  144.834631   Topic8  -5.0186  -0.5258
10       anime    0.449232  199.300071   Topic8  -5.0357  -0.8622
180     pretty    0.316180   59.232492   Topic8  -5.3870  -0.0001
33   character    0.182335  276.214837   Topic8  -5.9374  -2.0903

[595 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
634       1  0.425171  ability
634       2  0.106293  ability
634       4  0.106293  ability
634       5  0.212586  ability
634       7  0.212586  ability
...     ...       ...      ...
251       2  0.170637     work
251       3  0.113758     work
251       4  0.056879     work
251       5  0.056879     work
251       6  0.170637     work

[1108 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 5, 4, 3, 6, 1, 2, 7])

In [23]:
# Save the data frame as a CSV file
# job_datas.to_csv(r'D:\UMD Senior College Work\Spring 2021\INST447-0101\Programming Assignment 2\Chan_Matthew_INST447_PA2.csv')

In [24]:
# Week 9 Topic Modelling
# ner_corpus = ' '.join(ner_df['Name'])
# corp = open(r'D:\UMD Senior College Work\Spring 2021\INST447-0101\Week 9\corpus.txt',"w")
# corp.writelines(ner_corpus)
# corp.close()